In [ ]:
import gc
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, confusion_matrix,
                             mean_absolute_error, mean_squared_error)
from scipy.stats import pearsonr

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input, concatenate
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanAbsoluteError, MeanSquaredError
from tensorflow.keras import backend as K

# ——————————————————————————————
# Data loading and preprocessing
# ——————————————————————————————
with open('../image_names_data_fvc_fev_fevfvc_dlco_z_sexage_bmi_labels.pkl', 'rb') as f:
    (image_names, image_data, label_dict) = pickle.load(f)

print(label_dict.keys())

image_names = np.array(image_names)
image_data = np.array(image_data)
image_label = np.array(label_dict['fev1fvc_Z_label'])
sex_label = np.array(label_dict['sex_label'])
age_label = np.array(label_dict['age_label'])
bmi_label = np.array(label_dict['bmi_label'])

patients = np.array([x.split("_")[0] for x in image_names])
print("Number of Images:", len(patients))
print("Number of Unique Patients:", len(set(patients)))

# ——————————————————————————————
# Data Cleaning
# ——————————————————————————————
temp_names, temp_data, temp_label = [], [], []
temp_patients, temp_sex, temp_age, temp_bmi = [], [], [], []

for index in range(len(image_names)):
    if str(image_label[index]) in ['nv', 'None', 'nan']:
        continue
    if float(image_label[index]) > 5:
        continue
    if str(bmi_label[index]) in ['nan', 'None']:
        continue
    temp_names.append(image_names[index])
    temp_data.append(image_data[index])
    temp_label.append(float(image_label[index]))
    temp_patients.append(patients[index])
    temp_sex.append(sex_label[index])
    temp_age.append(age_label[index])
    temp_bmi.append(bmi_label[index])
    
image_names = np.array(temp_names)
image_data = np.array(temp_data)
image_label = np.array(temp_label)
patients = np.array(temp_patients)
sex_label = temp_sex
age_label = temp_age
bmi_label = temp_bmi

# Flatten image data (if needed for other processing)
image_data_flattened = image_data.reshape(image_data.shape[0], -1)
print("Shape of image_data after reshaping:", image_data_flattened.shape)
print("Sizes:", len(image_names), len(image_data), len(image_label))
print('Min:', np.min(image_label))
print('Mean:', np.mean(image_label))
print('Median:', np.percentile(image_label, 50))
print('75 percentile:', np.percentile(image_label, 90))
print('Max:', np.max(image_label))

total_people = len(image_label)
temp = [1 if x < -1.67 else 0 for x in image_label]
print("Less than -1.67", sum(temp)/total_people)

# histogram plot of image labels
plt.hist(image_label, bins=10)
plt.title("Histogram of Image Labels")
plt.xlabel("Image Label")
plt.ylabel("Frequency")
plt.show()

# ——————————————————————————————
# Data Reshaping and Preparation
# ——————————————————————————————
image_size = (256, 256)
batch_size = 32

# Expand dims and convert grayscale to RGB
image_data_channel_last = np.expand_dims(image_data, axis=3)
image_data_rgb_batch = np.repeat(image_data_channel_last, 3, axis=-1)

# Free memory
del image_data
gc.collect()

# Prepare additional input (sex, age, bmi)
sexagebmi_input = []
for index in range(len(sex_label)):
    s = 1 if sex_label[index] == "Male" else 0
    sexagebmi_input.append([s, int(age_label[index]), int(float(bmi_label[index]))])
sexagebmi_input = np.array(sexagebmi_input)
print("Shape of sexagebmi_input:", sexagebmi_input.shape)

# ——————————————————————————————
# Train-Test Split
# ——————————————————————————————
# Normalize the image data to [0, 1]
image_data_rgb_batch = image_data_rgb_batch / 255.0

# Select one image per unique patient
unique_patients = np.unique(patients)
selected_images_per_patient = {}
for patient_id in unique_patients:
    patient_image_indices = np.where(patients == patient_id)[0]
    selected_image_index = np.random.choice(patient_image_indices)
    selected_images_per_patient[patient_id] = selected_image_index
selected_image_indices = np.array(list(selected_images_per_patient.values()))

# Split selected indices into training and testing sets
train_indices, test_indices = train_test_split(
    selected_image_indices, 
    test_size=0.2, 
    random_state=42
)

# Create training and testing sets
x_train = image_data_rgb_batch[train_indices]
x_trainB = sexagebmi_input[train_indices]
y_train = image_label[train_indices]

x_test = image_data_rgb_batch[test_indices]
x_testB = sexagebmi_input[test_indices]
y_test = image_label[test_indices]

# Check for patient overlap between train and test sets
train_patient_ids = set(patients[train_indices])
test_patient_ids = set(patients[test_indices])
print("Overlap between train and test sets:", train_patient_ids & test_patient_ids)
if train_patient_ids & test_patient_ids:
    print("WARNING: There is overlap between train and test sets.")

# Plot distribution of true test values
plt.hist(y_test, bins=20, alpha=0.7, color='blue', label='True Values')
plt.axvline(0, color='red', linestyle='--', label='Threshold (0)')
plt.xlabel("True Values")
plt.ylabel("Frequency")
plt.title("Distribution of True Values")
plt.legend()
plt.show()

# ——————————————————————————————
# Model Definition
# ——————————————————————————————
# Branch A: Image Input with DenseNet121
inputA = Input(shape=(256, 256, 3))
base_model = DenseNet121(weights='../densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5', 
                         include_top=False, input_tensor=inputA)
for layer in base_model.layers[:-1]:
    layer.trainable = False
base_model.layers[-1].trainable = True
for layer in base_model.layers[-10:]:
    layer.trainable = True

x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='linear')(x)
branchA = Model(inputs=inputA, outputs=x)

# Branch B: Sex, Age, BMI Input
inputB = Input(shape=(3,))
y = Dense(9, activation="relu")(inputB)
branchB = Model(inputs=inputB, outputs=y)

# Combined model
combined = concatenate([branchA.output, branchB.output])
z = Dense(64, activation="relu")(combined)
z = Dropout(0.5)(z)
z = Dense(32, activation="relu")(z)
z = Dense(1, activation="linear")(z)

model = Model(inputs=[branchA.input, branchB.input], outputs=z)

# Custom weighted MAE loss function
def weighted_mae(y_true, y_pred):
    weights = tf.where(y_true > 0, 2.0, 1.0)
    return tf.reduce_mean(weights * tf.abs(y_true - y_pred))

# Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.9
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=weighted_mae,
              metrics=[MeanAbsoluteError(), MeanSquaredError()])

# ——————————————————————————————
# Model Training and Evaluation
# ——————————————————————————————
# Initialize metrics storage
mae_values = []
pearson_values = []
precision_values = []
recall_values = []
specificity_values = []
npv_values = []
accuracy_values = []

if "nv" in image_label and np.max(image_label) > 5:
    print("Skipping due to 'nv' category with values greater than 5.")
else:
    # Reassign training data using selected images per patient
    x_train = image_data_rgb_batch[selected_image_indices]
    x_trainB = sexagebmi_input[selected_image_indices]
    y_train = image_label[selected_image_indices]

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(x=[x_train, x_trainB], y=y_train,
                        batch_size=batch_size,
                        epochs=100,
                        verbose=1,
                        validation_data=([x_test, x_testB], y_test),
                        callbacks=[early_stopping])

    # Make predictions on the test set
    predictions = model.predict([x_test, x_testB])
    residuals = y_test - predictions.flatten()
    
    # Plot residuals (single consolidated plot)
    plt.scatter(y_test, residuals, alpha=0.5, label='Residuals')
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("True Values")
    plt.ylabel("Residuals (True - Predicted)")
    plt.title("Residuals vs True Values")
    plt.legend()
    plt.show()
    
    # Calculate binary labels for the lowest 10th percentile using dynamic threshold
    z_score_value = np.percentile(y_test, 10)
    true_value_binary_is_lowest = [1 if value <= z_score_value else 0 for value in y_test]
    predict_value_binary_is_lowest = [1 if value <= z_score_value else 0 for value in predictions.flatten()]

    correct_predictions = sum(1 for true_val, pred_val in zip(true_value_binary_is_lowest, predict_value_binary_is_lowest)
                              if true_val == pred_val)
    accuracy_lowest_10th_percentile = correct_predictions / len(true_value_binary_is_lowest)
    print(f"Accuracy for the lowest 10th percentile: {accuracy_lowest_10th_percentile:.2%}")

    # Calculate and print regression metrics
    mae = mean_absolute_error(y_test, predictions)
    pearson_corr, _ = pearsonr(y_test, predictions.flatten())
    print(f"Mean Absolute Error: {mae:.2f}, Pearson Correlation: {pearson_corr:.2f}")

    confusion_mat = confusion_matrix(true_value_binary_is_lowest, predict_value_binary_is_lowest)
    tn, fp, fn, tp = confusion_mat.ravel()
    
    plt.figure(figsize=(6, 4))
    sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Not Lowest 10%', 'Lowest 10%'],
                yticklabels=['Not Lowest 10%', 'Lowest 10%'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
    
    precision = precision_score(true_value_binary_is_lowest, predict_value_binary_is_lowest)
    recall = recall_score(true_value_binary_is_lowest, predict_value_binary_is_lowest)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    accuracy_metric = accuracy_score(true_value_binary_is_lowest, predict_value_binary_is_lowest)
    
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, Specificity: {specificity:.2f}, NPV: {npv:.2f}, Accuracy: {accuracy_metric:.2f}")
    
    # Evaluate performance separately for true values > 0 and <= 0
    positive_indices = np.where(y_test > 0)[0]
    negative_indices = np.where(y_test <= 0)[0]
    
    mae_positive = mean_absolute_error(y_test[positive_indices], predictions.flatten()[positive_indices])
    mae_negative = mean_absolute_error(y_test[negative_indices], predictions.flatten()[negative_indices])
    print(f"MAE for True Values > 0: {mae_positive:.2f}")
    print(f"MAE for True Values <= 0: {mae_negative:.2f}")
    
    # Scatter plot: Predicted vs True values for different true value ranges
    plt.scatter(y_test[positive_indices], predictions.flatten()[positive_indices], color='green', alpha=0.6, label='True > 0')
    plt.scatter(y_test[negative_indices], predictions.flatten()[negative_indices], color='blue', alpha=0.6, label='True <= 0')
    plt.plot([-5, 5], [-5, 5], color='red', linestyle='--', label='Perfect Prediction Line')
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title("Predicted vs True Values (Separated by True Value Sign)")
    plt.legend()
    plt.show()
    
    # Overall scatter plot for test data
    plt.scatter(y_test, predictions.flatten())
    plt.xlabel("True values")
    plt.ylabel("Predicted values")
    plt.title("True vs Predicted Values (Test Data)")
    plt.show()
    
    # Append metrics for later analysis
    mae_values.append(mae)
    pearson_values.append(pearson_corr)
    precision_values.append(precision)
    recall_values.append(recall)
    specificity_values.append(specificity)
    npv_values.append(npv)
    accuracy_values.append(accuracy_metric)
    
    del model, x_train, x_test, y_train, y_test
    gc.collect()
    K.clear_session()

# ——————————————————————————————
# Plot Training and Validation Loss
# ——————————————————————————————
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

# ——————————————————————————————
# Average Metrics Reporting
# ——————————————————————————————
average_mae = round(sum(mae_values) / len(mae_values), 2)
average_pearson = round(sum(pearson_values) / len(pearson_values), 2)
print(f"\nAverage Mean Absolute Error: {average_mae:.2f}, Average Pearson Correlation: {average_pearson:.2f}")

average_precision = round(sum(precision_values) / len(precision_values), 2)
average_recall = round(sum(recall_values) / len(recall_values), 2)
average_specificity = round(sum(specificity_values) / len(specificity_values), 2)
average_npv = round(sum(npv_values) / len(npv_values), 2)
average_accuracy = round(sum(accuracy_values) / len(accuracy_values), 2)
print(f"\nAverage Precision: {average_precision:.2f}, Average Recall: {average_recall:.2f}, "
      f"Average Specificity: {average_specificity:.2f}, Average NPV: {average_npv:.2f}, "
      f"Average Accuracy: {average_accuracy:.2f}")